In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

/home/bax/anaconda3/envs/ko3.7/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence
/home/bax/anaconda3/envs/ko3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:18: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
/home/bax/anaconda3/envs/ko3.7/lib/python3.7/site-packages/sklearn/model_selection/_search.py:16: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, namedtuple, defaultdict, Sequence


In [2]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [3]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
# device = torch.device("cpu")
device = torch.device("cuda")

In [5]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /mnt/c/Users/BAX/desktop/psb/biz/kobert_ori/.cache/kobert_v1.zip
using cached model. /mnt/c/Users/BAX/desktop/psb/biz/kobert_ori/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [6]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /mnt/c/Users/BAX/desktop/psb/biz/kobert_ori/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [7]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)



In [22]:
PATH = './model_parameter_score/'
model_01 = torch.load(PATH + 'KeyActivities01.pt')
model_01.load_state_dict(torch.load(PATH + 'KeyActivities01_state_dict.pth'))

model_02 = torch.load(PATH + 'KeyActivities02.pt')
model_02.load_state_dict(torch.load(PATH + 'KeyActivities02_state_dict.pth'))

model_03 = torch.load(PATH + 'KeyActivities03.pt')
model_03.load_state_dict(torch.load(PATH + 'KeyActivities03_state_dict.pth'))

max_len = 300

In [23]:
# model.eval()

In [24]:
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

def testModel(model, seq):
    cate = [0,1,2,3,4,5]
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
    tokenized = transform(tmp)

    model.eval()
    result = model(torch.tensor([tokenized[0]]).to(device).long(), [tokenized[1]], torch.tensor(tokenized[2]).to(device).long())
    idx = result.argmax().cpu().item()
    # print("점수는", cate[idx] * 16)
    # print("신뢰도는:", "{:.2f}%".format(softmax(result,idx)))
    return cate[idx]

In [47]:
# testModel(model,"")
sequence = "자동차 업계관련 된 협력업체 및 파트너를 모집하여 \n실적유발시 커미션을 지급하고 이에 만족감을 느끼게 하여 \n홍보도 병행할 수 있게 한다"
key01 = testModel(model_01, sequence)
key02 = testModel(model_02, sequence)
key03 = testModel(model_03, sequence)

print('-'*50)
print(f'입력된 문장 : {sequence}')
print('-'*50)
print(f'1. 계획서의 핵심자원은 경쟁자들이 복제하기 어려운가?   {key01*16}/80')
print(f'2. 자원 수요가 예측 가능한가?                         {key02*2}/10')
print(f'3. 계획서의 핵심자원을 적재적소에 효율적으로 배치했는가? {key03*2}/10')
print(f'KeyActivity 총점 : {key01*16+key02*2+key03*2}/100')

--------------------------------------------------
입력된 문장 : 자동차 업계관련 된 협력업체 및 파트너를 모집하여 
실적유발시 커미션을 지급하고 이에 만족감을 느끼게 하여 
홍보도 병행할 수 있게 한다
--------------------------------------------------
1. 계획서의 핵심자원은 경쟁자들이 복제하기 어려운가?   48/80
2. 자원 수요가 예측 가능한가?                         4/10
3. 계획서의 핵심자원을 적재적소에 효율적으로 배치했는가? 4/10
KeyActivity 총점 : 56/100
